In [1]:

import win32com.client


In [2]:

# 연결 여부 체크
objCpCybos = win32com.client.Dispatch("CpUtil.CpCybos")
bConnect = objCpCybos.IsConnect
if (bConnect == 0):
    print("PLUS가 정상적으로 연결되지 않음. ")
    exit()

In [3]:

# 종목코드 리스트 구하기
objCpCodeMgr = win32com.client.Dispatch("CpUtil.CpCodeMgr")
codeList = objCpCodeMgr.GetStockListByMarket(1)  # 거래소
codeList2 = objCpCodeMgr.GetStockListByMarket(2)  # 코스닥
print('거래소 전체 종목 : {}'.format(len(codeList)))
print('코스닥 전체 종목 : {}'.format(len(codeList2)))
print("거래소 + 코스닥 종목코드 ", len(codeList) + len(codeList2))

#secondcode, name, stdprice API

# for i, code in enumerate(codeList):
#     secondCode = objCpCodeMgr.GetStockSectionKind(code)
#     name = objCpCodeMgr.CodeToName(code)
#     stdPrice = objCpCodeMgr.GetStockStdPrice(code)
#     print(i, code, secondCode, stdPrice, name)




거래소 전체 종목 : 1575
코스닥 전체 종목 : 1451
거래소 + 코스닥 종목코드  3026


In [4]:
#코스닥 기준 dictionary

print("코스닥 종목코드", len(codeList2))
kosdaq_name_dict = {}
kosdaq_2nd_cd_dict = {}
for i, code in enumerate(codeList2):
    kosdaq_name_dict[code] = objCpCodeMgr.CodeToName(code)
    kosdaq_2nd_cd_dict[code] = objCpCodeMgr.GetStockSectionKind(code)


코스닥 종목코드 1451


In [5]:
#전체 코스닥 기업명

kosdaq_enpnm = list(kosdaq_name_dict.values())

In [ ]:
'''
당일 상승률 상위 200 종목 실시간 통신 예제


# 설명: 당일 상승률 상위 200 종목을 가져와 현재가  실시간 조회하는 샘플
# CpEvent: 실시간 현재가 수신 클래스
# CpStockCur : 현재가 실시간 통신 클래스
# Cp7043 : 상승률 상위 종목 통신 서비스 - 연속 조회를 통해 200 종목 가져옴
# CpMarketEye: 복수 종목 조회 서비스 - 200 종목 현재가를 조회 함.

'''


In [6]:
import sys
from PyQt5.QtWidgets import *
import win32com.client

In [7]:

# CpEvent: 실시간 이벤트 수신 클래스
class CpEvent:
    def set_params(self, client):
        self.client = client
 
    def OnReceived(self):
        code = self.client.GetHeaderValue(0)  # 코드
        name = self.client.GetHeaderValue(1)  # 명
        timess = self.client.GetHeaderValue(18)  # 초
        exFlag = self.client.GetHeaderValue(19)  # 예상체결 플래그
        cprice = self.client.GetHeaderValue(13)  # 현재가
        diff = self.client.GetHeaderValue(2)  # 대비
        cVol = self.client.GetHeaderValue(17)  # 순간체결수량
        vol = self.client.GetHeaderValue(9)  # 거래량
 
        if (exFlag == ord('1')):  # 동시호가 시간 (예상체결)
            print("실시간(예상체결)", name, timess, "*", cprice, "대비", diff, "체결량", cVol, "거래량", vol)
        elif (exFlag == ord('2')):  # 장중(체결)
            print("실시간(장중 체결)", name, timess, cprice, "대비", diff, "체결량", cVol, "거래량", vol)
 

In [8]:

# CpStockCur: 실시간 현재가 요청 클래스
class CpStockCur:
    def Subscribe(self, code):
        self.objStockCur = win32com.client.Dispatch("DsCbo1.StockCur")
        handler = win32com.client.WithEvents(self.objStockCur, CpEvent)
        self.objStockCur.SetInputValue(0, code)
        handler.set_params(self.objStockCur)
        self.objStockCur.Subscribe()
 
    def Unsubscribe(self):
        self.objStockCur.Unsubscribe()
 

In [9]:

# Cp7043 상승률 상위 요청 클래스
class Cp7043:
    def __init__(self):
        # 통신 OBJECT 기본 세팅
        self.objRq = win32com.client.Dispatch("CpSysDib.CpSvrNew7043")
        self.objRq.SetInputValue(0, ord('0')) # 거래소 + 코스닥
        self.objRq.SetInputValue(1, ord('2'))  # 상승
        self.objRq.SetInputValue(2, ord('1'))  # 당일
        self.objRq.SetInputValue(3, 21)  # 전일 대비 상위 순
        self.objRq.SetInputValue(4, ord('1'))  # 관리 종목 제외
        self.objRq.SetInputValue(5, ord('0'))  # 거래량 전체
        self.objRq.SetInputValue(6, ord('0'))  # '표시 항목 선택 - '0': 시가대비
        self.objRq.SetInputValue(7, 0)  #  등락율 시작
        self.objRq.SetInputValue(8, 30)  # 등락율 끝
 
    # 실제적인 7043 통신 처리
    def rq7043(self, retcode):
        self.objRq.BlockRequest()
        # 현재가 통신 및 통신 에러 처리
        rqStatus = self.objRq.GetDibStatus()
        rqRet = self.objRq.GetDibMsg1()
        print("통신상태", rqStatus, rqRet)
        if rqStatus != 0:
            return False
 
        cnt = self.objRq.GetHeaderValue(0)
        cntTotal  = self.objRq.GetHeaderValue(1)
        print(cnt, cntTotal)
 
        for i in range(cnt):
            code = self.objRq.GetDataValue(0, i)  # 코드
            retcode.append(code)
            if len(retcode) >=  200:       # 최대 200 종목만,
                break
            name = self.objRq.GetDataValue(1, i)  # 종목명
            diffflag = self.objRq.GetDataValue(3, i)
            diff = self.objRq.GetDataValue(4, i)
            vol = self.objRq.GetDataValue(6, i)  # 거래량
            print(code, name, diffflag, diff, vol)
 
    def Request(self, retCode):
        self.rq7043(retCode)
 
        # 연속 데이터 조회 - 200 개까지만.
        while self.objRq.Continue:
            self.rq7043(retCode)
            print(len(retCode))
            if len(retCode) >= 200:
                break
 
        # #7043 상승하락 서비스를 통해 받은 상승률 상위 200 종목
        size = len(retCode)
        for i in range(size):
            print(retCode[i])
        return True

In [10]:

# CpMarketEye : 복수종목 현재가 통신 서비스
class CpMarketEye:
    def Request(self, codes, rqField):
        # 연결 여부 체크
        objCpCybos = win32com.client.Dispatch("CpUtil.CpCybos")
        bConnect = objCpCybos.IsConnect
        if (bConnect == 0):
            print("PLUS가 정상적으로 연결되지 않음. ")
            return False
 
        # 관심종목 객체 구하기
        objRq = win32com.client.Dispatch("CpSysDib.MarketEye")
        # 요청 필드 세팅 - 종목코드, 종목명, 시간, 대비부호, 대비, 현재가, 거래량
        # rqField = [0,17, 1,2,3,4,10]
        objRq.SetInputValue(0, rqField) # 요청 필드
        objRq.SetInputValue(1, codes)  # 종목코드 or 종목코드 리스트
        objRq.BlockRequest()
 
 
        # 현재가 통신 및 통신 에러 처리
        rqStatus = objRq.GetDibStatus()
        rqRet = objRq.GetDibMsg1()
        print("통신상태", rqStatus, rqRet)
        if rqStatus != 0:
            return False
 
        cnt  = objRq.GetHeaderValue(2)
 
        for i in range(cnt):
            rpCode = objRq.GetDataValue(0, i)  # 코드
            rpName = objRq.GetDataValue(1, i)  # 종목명
            rpTime= objRq.GetDataValue(2, i)  # 시간
            rpDiffFlag = objRq.GetDataValue(3, i)  # 대비부호
            rpDiff = objRq.GetDataValue(4, i)  # 대비
            rpCur = objRq.GetDataValue(5, i)  # 현재가
            rpVol = objRq.GetDataValue(6, i)  # 거래량
            print(rpCode, rpName, rpTime,  rpDiffFlag, rpDiff, rpCur, rpVol)
 
        return True
 
 
 

In [11]:

class MyWindow(QMainWindow):
 
    def __init__(self):
        super().__init__()
        self.setWindowTitle("PLUS API TEST")
        self.setGeometry(300, 300, 300, 180)
        self.isSB = False
        self.objCur = []
 
        btnStart = QPushButton("요청 시작", self)
        btnStart.move(20, 20)
        btnStart.clicked.connect(self.btnStart_clicked)
 
        btnStop = QPushButton("요청 종료", self)
        btnStop.move(20, 70)
        btnStop.clicked.connect(self.btnStop_clicked)
 
        btnExit = QPushButton("종료", self)
        btnExit.move(20, 120)
        btnExit.clicked.connect(self.btnExit_clicked)
 
    def StopSubscribe(self):
        if self.isSB:
            cnt = len(self.objCur)
            for i in range(cnt):
                self.objCur[i].Unsubscribe()
            print(cnt, "종목 실시간 해지되었음")
        self.isSB = False
 
        self.objCur = []
 
    def btnStart_clicked(self):
        self.StopSubscribe();
        codes = []
        obj7043 = Cp7043()
        if obj7043.Request(codes) == False:
            return
 
        print("상승종목 개수:", len(codes))
 
        # 요청 필드 배열 - 종목코드, 시간, 대비부호 대비, 현재가, 거래량, 종목명
        rqField = [0, 1, 2, 3, 4, 10, 17]  #요청 필드
        objMarkeyeye = CpMarketEye()
        if (objMarkeyeye.Request(codes, rqField) == False):
            exit()
 
        cnt = len(codes)
        for i in range(cnt):
            self.objCur.append(CpStockCur())
            self.objCur[i].Subscribe(codes[i])
 
        print("빼기빼기================-")
        print(cnt , "종목 실시간 현재가 요청 시작")
        self.isSB = True
 
    def btnStop_clicked(self):
        self.StopSubscribe()
 
 
    def btnExit_clicked(self):
        self.StopSubscribe()
        exit()
 


In [12]:
if __name__ == "__main__":
    app = QApplication(sys.argv)
    myWindow = MyWindow()
    myWindow.show()
    app.exec_()


통신상태 0 
40 914
A195990 에이비프로바이오 49 580 268356706
A086040 바이오톡스텍 49 3700 22172351
A033180 필룩스 49 985 37370041
A084990 헬릭스미스 49 6900 4127878
A036090 위지트 49 294 135513245
A032820 우리기술 49 360 55326595
A032850 비트컴퓨터 49 2920 23174111
A033270 유나이티드제약 50 11700 10648834
A032620 유비케어 50 2210 19725160
A030790 비케이탑스 50 1700 2424066
A041020 인프라웨어 50 265 1614180
A082850 우리바이오 50 920 33744366
A127120 디엔에이링크 50 2200 2999507
A007460 에이프로젠 KIC 50 325 109583637
A083640 인콘 50 405 16444044
A018880 한온시스템 50 1950 14470064
A089530 에이티세미콘 50 111 17587538
A095700 제넥신 50 15000 2627981
A290510 코리아센터 50 1300 18068104
A033230 인성정보 50 420 7686353
A123860 아나패스 50 2850 490337
A042110 에스씨디 50 175 36635773
A069920 아이에스이커머스 50 445 680720
A139050 데일리블록체인 50 155 66927495
A011280 태림포장 50 600 20102103
A131290 티에스이 50 5900 296763
A269620 시스웍 50 385 21604366
A006370 대구백화점 50 1100 316647
A016920 카스 50 260 24920775
A215380 우정바이오 50 1100 19574810
A051160 지어소프트 50 1250 1243446
A285490 노바텍 50 5050 389852
A294090 이오플로우 50 4100 10621

빼기빼기================-
200 종목 실시간 현재가 요청 시작
